# E-Commerce Exploratory Data Analysis

This notebook analyzes e-commerce sales data to surface key business metrics
around revenue, product performance, geographic distribution, and customer
experience. All analysis is driven by a **configurable date range** so the
same notebook can be re-run for any period.

## Table of Contents

1. [Configuration](#1-configuration)
2. [Data Loading and Preparation](#2-data-loading-and-preparation)
3. [Data Dictionary](#3-data-dictionary)
4. [Revenue Analysis](#4-revenue-analysis)
5. [Product Category Analysis](#5-product-category-analysis)
6. [Geographic Analysis](#6-geographic-analysis)
7. [Customer Experience Analysis](#7-customer-experience-analysis)
8. [Summary of Observations](#8-summary-of-observations)

---

## 1. Configuration

Set the analysis and comparison periods below. Change these values to
re-run the entire notebook for a different time window.

In [ ]:
# -- Analysis parameters (edit these to change the analysis period) ----------
ANALYSIS_YEAR = 2023
COMPARISON_YEAR = 2022

DATA_DIR = "ecommerce_data"

# Consistent colour palette used across all charts
COLOR_PRIMARY = "#2C6E91"
COLOR_SECONDARY = "#E8804C"
COLOR_PALETTE = ["#2C6E91", "#E8804C", "#6BAA75", "#E06C78",
                 "#8E7CC3", "#C9A84C", "#4CBAE8", "#D45B90",
                 "#5BB5A2", "#A67C52", "#7E57C2", "#FF8A65",
                 "#4DB6AC"]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import plotly.express as px

import data_loader as dl
import business_metrics as bm

# Matplotlib defaults for a clean, consistent look
plt.rcParams.update({
    "figure.figsize": (10, 5),
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
})

---
## 2. Data Loading and Preparation

Load the six CSV datasets, parse date columns, merge order items with
orders, and filter to delivered orders only.

In [ ]:
datasets = dl.load_datasets(DATA_DIR)

orders      = dl.parse_order_dates(datasets["orders"])
order_items = datasets["order_items"]
products    = datasets["products"]
customers   = datasets["customers"]
reviews     = datasets["reviews"]
payments    = datasets["payments"]

print(f"Orders:      {len(orders):,} rows")
print(f"Order items: {len(order_items):,} rows")
print(f"Products:    {products['product_id'].nunique():,} unique")
print(f"Categories:  {products['product_category_name'].nunique()}")
print(f"Customers:   {len(customers):,} rows")
print(f"Reviews:     {len(reviews):,} rows")

In [ ]:
# Build merged sales data and filter to delivered orders
sales_data = dl.build_sales_data(order_items, orders)
delivered_all = dl.filter_delivered(sales_data)

# Add delivery speed (days from purchase to customer delivery)
delivered_all = dl.add_delivery_speed(delivered_all)

# Split into analysis and comparison periods
delivered_current  = dl.filter_by_year(delivered_all, ANALYSIS_YEAR)
delivered_previous = dl.filter_by_year(delivered_all, COMPARISON_YEAR)

print(f"Delivered items -- {ANALYSIS_YEAR}: {len(delivered_current):,}  |  "
      f"{COMPARISON_YEAR}: {len(delivered_previous):,}")

In [ ]:
delivered_current.head(3)

In [ ]:
order_items.describe()

---
## 3. Data Dictionary

| Dataset | Key Columns | Description |
|---------|------------|-------------|
| **orders** | order_id, customer_id, order_status, order_purchase_timestamp, order_delivered_customer_date | One row per order with lifecycle timestamps |
| **order_items** | order_id, order_item_id, product_id, seller_id, price, freight_value | One row per item in an order; price is the item sale price |
| **products** | product_id, product_category_name | Product catalogue with category and physical attributes |
| **customers** | customer_id, customer_unique_id, customer_city, customer_state | Customer location details |
| **reviews** | review_id, order_id, review_score | Customer review per order (score 1-5) |
| **payments** | order_id, payment_type, payment_installments, payment_value | Payment method and value per order |

**Key business terms**

- **Revenue** -- sum of item `price` for delivered orders (excludes freight).
- **Average Order Value (AOV)** -- mean of per-order revenue.
- **Month-over-month (MoM) growth** -- percentage change in monthly revenue compared to the prior month.
- **Delivery days** -- calendar days from purchase to customer delivery.

---
## 4. Revenue Analysis

This section covers total revenue, year-over-year growth, average order
value, order counts, and the monthly revenue trend.

### 4.1 Total Revenue and Year-over-Year Growth

In [ ]:
rev_current  = bm.total_revenue(delivered_current)
rev_previous = bm.total_revenue(delivered_previous)
rev_change   = bm.revenue_growth(delivered_current, delivered_previous)

print(f"Total revenue in {ANALYSIS_YEAR}: ${rev_current:,.2f}")
print(f"Total revenue in {COMPARISON_YEAR}: ${rev_previous:,.2f}")
print(f"Year-over-year change: {rev_change * 100:.2f}%")

### 4.2 Month-over-Month Revenue Growth

In [ ]:
mom_growth = bm.month_over_month_growth(delivered_current)
avg_mom    = bm.average_mom_growth(delivered_current)

print(f"Average month-over-month growth in {ANALYSIS_YEAR}: {avg_mom * 100:.2f}%")
print()
print(mom_growth.to_string())

### 4.3 Average Order Value

In [ ]:
aov_current  = bm.average_order_value(delivered_current)
aov_change   = bm.aov_growth(delivered_current, delivered_previous)

print(f"Average order value in {ANALYSIS_YEAR}: ${aov_current:,.2f}")
print(f"Compared to {COMPARISON_YEAR}: {aov_change * 100:.2f}%")

### 4.4 Total Orders

In [ ]:
orders_current  = bm.total_orders(delivered_current)
orders_change   = bm.order_count_growth(delivered_current, delivered_previous)

print(f"Total delivered orders in {ANALYSIS_YEAR}: {orders_current:,}")
print(f"Compared to {COMPARISON_YEAR}: {orders_change * 100:.2f}%")

### 4.5 Monthly Revenue Trend

In [ ]:
monthly_rev = bm.monthly_revenue(delivered_current)

fig, ax = plt.subplots()
ax.plot(monthly_rev["month"], monthly_rev["revenue"],
        marker="o", color=COLOR_PRIMARY, linewidth=2)
ax.set_title(f"Monthly Revenue Trend ({ANALYSIS_YEAR})")
ax.set_xlabel("Month")
ax.set_ylabel("Revenue (USD)")
ax.set_xticks(range(1, 13))
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

### 4.6 Order Status Distribution

In [ ]:
status_dist = bm.order_status_distribution(orders, ANALYSIS_YEAR)
print(f"Order status distribution in {ANALYSIS_YEAR}:")
print()
for status, proportion in status_dist.items():
    print(f"  {status:<14s} {proportion:>7.2%}")

---
## 5. Product Category Analysis

Revenue breakdown by product category for the analysis period.

In [ ]:
category_revenue = bm.revenue_by_category(delivered_current, products)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(range(len(category_revenue)), category_revenue.values,
              color=COLOR_PALETTE[:len(category_revenue)])
ax.set_xticks(range(len(category_revenue)))
ax.set_xticklabels(category_revenue.index, rotation=45, ha="right")
ax.set_title(f"Revenue by Product Category ({ANALYSIS_YEAR})")
ax.set_xlabel("Product Category")
ax.set_ylabel("Revenue (USD)")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6. Geographic Analysis

Revenue distribution across US states for the analysis period.

In [ ]:
state_revenue = bm.revenue_by_state(delivered_current, orders, customers)

fig = px.choropleth(
    state_revenue,
    locations="customer_state",
    color="revenue",
    locationmode="USA-states",
    scope="usa",
    title=f"Revenue by State ({ANALYSIS_YEAR})",
    color_continuous_scale=[
        [0, "#d4e8f0"],
        [0.5, "#5a9ec2"],
        [1, "#1a3a50"],
    ],
    labels={"revenue": "Revenue (USD)"},
)
fig.update_layout(geo=dict(lakecolor="white"))
fig.show()

---
## 7. Customer Experience Analysis

Relationship between delivery speed and customer review scores,
overall review score distribution, and average delivery time.

In [ ]:
review_summary = bm.review_delivery_summary(delivered_current, reviews)

### 7.1 Average Review Score by Delivery Day

In [ ]:
by_day = bm.avg_review_by_delivery_day(review_summary)
by_day

### 7.2 Average Review Score by Delivery Bucket

In [ ]:
by_bucket = bm.avg_review_by_delivery_bucket(review_summary)
by_bucket

### 7.3 Overall Review Score and Delivery Time

In [ ]:
avg_review = bm.average_review_score(review_summary)
avg_delivery = bm.average_delivery_days(review_summary)

print(f"Average review score in {ANALYSIS_YEAR}: {avg_review:.2f} / 5")
print(f"Average delivery time in {ANALYSIS_YEAR}: {avg_delivery:.1f} days")

### 7.4 Review Score Distribution

In [ ]:
score_dist = bm.review_score_distribution(review_summary)

fig, ax = plt.subplots(figsize=(8, 4))
ax.barh(
    score_dist.index.astype(str),
    score_dist.values,
    color=COLOR_PRIMARY,
)
ax.set_title(f"Review Score Distribution ({ANALYSIS_YEAR})")
ax.set_xlabel("Proportion of Reviews")
ax.set_ylabel("Review Score")
ax.xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1))
ax.grid(axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 8. Summary of Observations

The cell below dynamically summarises the computed metrics for the
configured analysis period.

In [ ]:
print(f"=== E-Commerce KPI Summary: {ANALYSIS_YEAR} vs {COMPARISON_YEAR} ===")
print()
print(f"Revenue")
print(f"  Total revenue ({ANALYSIS_YEAR}):        ${rev_current:>12,.2f}")
print(f"  Year-over-year revenue change:  {rev_change * 100:>+10.2f}%")
print(f"  Avg month-over-month growth:    {avg_mom * 100:>+10.2f}%")
print()
print(f"Orders")
print(f"  Total delivered orders:         {orders_current:>10,}")
print(f"  Year-over-year order change:    {orders_change * 100:>+10.2f}%")
print(f"  Average order value:            ${aov_current:>10,.2f}")
print(f"  AOV year-over-year change:      {aov_change * 100:>+10.2f}%")
print()
print(f"Customer Experience")
print(f"  Average review score:           {avg_review:>10.2f} / 5")
print(f"  Average delivery time:          {avg_delivery:>10.1f} days")
print()
top_category = category_revenue.index[0]
top_state = state_revenue.iloc[0]["customer_state"]
print(f"Top Performers")
print(f"  Highest-revenue category:       {top_category}")
print(f"  Highest-revenue state:          {top_state}")

---

## How to Use This Analysis

1. **Change the analysis period** -- edit `ANALYSIS_YEAR` and
   `COMPARISON_YEAR` in the Configuration cell and re-run all cells.
   For sub-year ranges, use `data_loader.filter_by_date_range()` instead of
   `filter_by_year()`.

2. **Reuse metric functions** -- import `business_metrics` and `data_loader`
   in any Python script or notebook:
   ```python
   import data_loader as dl
   import business_metrics as bm

   datasets = dl.load_datasets("ecommerce_data")
   orders = dl.parse_order_dates(datasets["orders"])
   # ... build, filter, compute
   ```

3. **Install dependencies** -- `pip install -r requirements.txt`